# toolmaker

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/langchain-tools_toolmaker.ipynb)

In [ ]:
# langchain-toolsのインストール
!pip install git+https://github.com/fuyu-quant/langchain-tools.git

In [ ]:
%%capture
!pip install langchain
!pip install openai

In [ ]:
# LangChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool


# langchaintools 
from langchaintools import toolmaker
from langchaintools import mltools


import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
tools = [Tool(name = 'toolmaker', func = toolmaker.toolmaker, description="")]

In [ ]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


tool_name = "XGBoost training"
agent.run(f"Please make a langchain-tool for {tool_name}")